# TCN Architectures and Variants (Current)

This notebook documents the TCN variants and their mathematical foundations.


## 1) TCN Block in This Codebase

Each `TCNBlock` has two causal dilated Conv1D layers plus residual connection.

For input sequence $x_t$:

$$
y_t^{(1)} = \sigma\left(\sum_{i=0}^{k-1} W_i^{(1)}x_{t-id} + b^{(1)}\right),
$$

$$
y_t^{(2)} = \sigma\left(\sum_{i=0}^{k-1} W_i^{(2)}y_{t-id}^{(1)} + b^{(2)}\right),
$$

$$
\text{out}_t = \text{ReLU}(y_t^{(2)} + r_t).
$$


## 2) Available TCN Variants

### Variant A: Plain TCN

- `actor_critic_type = "TCN"`
- `use_attention = False`
- `use_fusion = False`
- Actor/Critic: `TCNActor`, `TCNCritic`

### Variant B: TCN + Attention (toggle path)

- `actor_critic_type = "TCN"`
- `use_attention = True`
- Actor/Critic: `TCNAttentionActor`, `TCNAttentionCritic`

### Variant C: TCN + Fusion (toggle path)

- `actor_critic_type = "TCN"`
- `use_fusion = True`
- Uses fusion pathway implemented in current TCN stack.

### Variant D: TCN_ATTENTION (explicit alias)

- `actor_critic_type = "TCN_ATTENTION"`
- Same attention actor/critic classes as Variant B.


## 3) Attention Mathematics

After TCN feature extraction and projection, multi-head self-attention is applied:

$$
\text{Attention}(Q,K,V)=\text{softmax}\left(\frac{QK^\top}{\sqrt{d_k}}\right)V.
$$


## 4) Receptive Field Formula

This implementation has two Conv1D layers per block. With kernel size $k$ and dilation list $\{d_j\}$:

$$
\text{RF} = 1 + 2(k-1)\sum_j d_j.
$$

Current Phase 1 values:

- $k = 5$
- $d = [2, 4, 8]$

$$
\text{RF} = 1 + 2(5-1)(2+4+8) = 113.
$$


## 5) Dilation Implementation Detail

Blocks are created via:

- `zip(tcn_filters, tcn_dilations)`

So block count is:

$$
N_{\text{blocks}} = \min\left(|\text{tcn\_filters}|, |\text{tcn\_dilations}|\right).
$$

Current active values produce 3 blocks (`[32,64,64]` with `[2,4,8]`).


## 6) Dirichlet Head for TCN Variants

All TCN variants output Dirichlet concentration parameters $\alpha$ with configurable activation.

Current default:

- `dirichlet_alpha_activation = "elu"`.


In [ ]:
from src.config import PHASE1_CONFIG
ap = PHASE1_CONFIG['agent_params']
k = ap['tcn_kernel_size']
d = ap['tcn_dilations']
rf = 1 + 2*(k-1)*sum(d)
print('Current TCN setup')
print('  filters:', ap['tcn_filters'])
print('  kernel:', k)
print('  dilations:', d)
print('  sequence_length:', ap['sequence_length'])
print('  theoretical RF:', rf)
